In [ ]:
import pandas as pd
import seaborn as sns
import requests
import kaggle
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', 500)

How do I download using requests / API? <br>
https://github.com/Kaggle/kaggle-api <br>
kaggle datasets download -d christophercorrea/dc-residential-properties 

In [ ]:
#how do I not write the entire file path?
data = pd.read_csv('/Users/Julia/dc-residential-properties/DC_Properties.csv', low_memory=False)
print(data.shape)
print(data.columns.tolist())
data.head(10)

In [ ]:
ddat = pd.DataFrame.drop(data, columns = 'Unnamed: 0')

In [ ]:
#combine BR columns
ddat['BATHRMS'] = ddat['BATHRM'] + ddat['HF_BATHRM']
ddat = pd.DataFrame.drop(ddat, columns = ['BATHRM', 'HF_BATHRM'])

In [ ]:
#percentages of NaN values per column
perc_nan=(ddat.isnull().sum(axis = 0))/len(ddat)
#delete NaN rows from columns with few NaN values:
dlist = (perc_nan[perc_nan <= 0.02].index).tolist()
print(dlist)
ddat = ddat.dropna(subset=dlist)


In [ ]:
#evaluate rows with more NaN values
print(perc_nan[perc_nan > 0.02])

In [ ]:
#Are values the same for CITY and STATE?
print(data.CITY.value_counts())
print(data.STATE.value_counts())

In [ ]:
ddat = pd.DataFrame.drop(ddat, columns=['CMPLX_NUM', 'STATE', 'CITY', 'FULLADDRESS', 'NATIONALGRID'])
print(ddat.shape)
print(ddat.columns.tolist())
ddat.head(10)


In [ ]:
#We will make a small dataset, which contains no null values from price. 
sdat = ddat.dropna(subset=['PRICE'])
print(sdat.shape)
sdat.head(10)

In [ ]:
#if we remove all NaNs, we have no data.
ldat = sdat.dropna()
ldat.head()

In [ ]:
#types of data in df
sdat.dtypes

In [ ]:
#TWO LISTS OF COLUMNS. ONE CATEGORICAL, ONE NUMERICAL catcol and numcol
#TWO DFS of categorical and numerical. catdf and numdf
catcol = []
numcol = []
for col in sdat.columns:
    if (sdat[col].dtype == object):
        catcol.append(col)
    if (sdat[col].dtype == np.int64) or (sdat[col].dtype == np.float64):
        numcol.append(col)
catdf = sdat[catcol].astype('category')
numdf = sdat[numcol].astype(np.float64)
print(catcol)
print(numcol)

In [ ]:
#show information about values, outlier counts, and fences per numerical variable
def show_outliers(df_in, col):
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col] < fence_low) | (df_in[col] > fence_high)]
    print(col)
    print("fence_low:", fence_low) 
    print("Q1:", q1)
    print("Q3:", q3) 
    print("fence_high:", fence_high)
    print("num_outliers:", len(df_out))
    print((df_in[col].value_counts().sort_index()).head(5))
    print((df_in[col].value_counts().sort_index()).tail(5))
for col in numdf.columns:
    show_outliers(numdf, col)

In [ ]:
def perc_outliers(df_in, col, slow, shigh):
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-slow*iqr
    fence_high = q3+shigh*iqr
    df_out = df_in.loc[(df_in[col] < fence_low) | (df_in[col] > fence_high)]
    print("Percentage of outliers:", len(df_out)/len(df_in))

In [ ]:
print(perc_outliers(sdat, 'LIVING_GBA', 1.5, 1.5))
sns.boxplot(x=sdat['LIVING_GBA'], whis=1.5)

In [ ]:
sns.distplot(sdat['LIVING_GBA'].dropna())

In [ ]:
print(perc_outliers(sdat, 'GBA', 1.5, 1.5))
sns.boxplot(x=sdat['GBA'], whis=1.5)

In [ ]:
sns.distplot(sdat['GBA'].dropna())

In [ ]:
print(perc_outliers(sdat, 'PRICE', 1, 2.5))
sns.boxplot(x=sdat['PRICE'], whis=2.5)

In [ ]:
sns.distplot(sdat['PRICE'].dropna())

In [ ]:
print(perc_outliers(sdat, 'LANDAREA', 1, 2.5))
sns.boxplot(x=sdat['LANDAREA'], whis=2.5)

In [ ]:
sns.distplot(sdat['LANDAREA'].dropna())

In [ ]:
#checking values for categorical:
for col in catdf.columns:
    print(col) 
    print(catdf[col].value_counts().sort_index())

In [ ]:
#REMOVING OUTLIERS: When limiting these values, the data shrunk considerably. 
#The only data that was likely faulty (and not just out of "the norm") was
#STORIES: 250.00, 275.00, 826.00        
#YR_RMDL: 20

odat = sdat.loc[(sdat['NUM_UNITS'] >= 1) & (sdat['NUM_UNITS'] <= 4) & (sdat['ROOMS'] <= 12) & (sdat['BEDRM'] <= 6) & (sdat['YR_RMDL'] >= 1880) & 
          (sdat['STORIES'] >= 1) & (sdat['STORIES'] <= 3) & (sdat['KITCHENS'] >= 1) & (sdat['KITCHENS'] <= 4) & (sdat['FIREPLACES'] >= 1) & 
          (sdat['FIREPLACES'] <= 3) & (sdat['BATHRMS'] >= 1) & (sdat['BATHRMS'] <= 7)]

odat2 = sdat.loc[((sdat['STORIES'] < 30) | (sdat['STORIES'].isna()))  & ((sdat['YR_RMDL'] >= 1880) | (sdat['YR_RMDL'].isna()))]

print(sdat.shape)
print(odat2.shape)

In [ ]:
#Removeing outliers from GBA, LIVING_GBA, LANDAREA, and PRICE

takind = []
def remove_outliers (df_in, col, slow, shigh):
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-slow*iqr
    fence_high = q3+shigh*iqr
    global iout
    iout = (df_in.loc[(df_in[col] < fence_low) | (df_in[col] > fence_high)]).index.tolist()
    for i in iout:
        if i not in takind:
            takind.append(i) 
    
remove_outliers(odat2, 'GBA', 1.5, 1.5)
remove_outliers(odat2, 'LIVING_GBA', 1.5, 1.5)
remove_outliers(odat2, 'LANDAREA', 1, 2.5)
remove_outliers(odat2, 'PRICE', 1, 2.5)
print(len(takind))

In [ ]:
odatd = odat2.drop(index=takind)
odatd.shape
odatd.head()

In [78]:
#percentage of missing data per row in new dataset
def missingdat(df):
    percnan=(df.isnull().sum(axis=0))/len(df)
    nulcolstat = percnan[percnan>0]
    nullist = (nulcolstat.index).tolist()
    nuldat = sdat[nullist]
    print(nulcolstat)
print(missingdat(odatd))

HEAT                  0.000000
AC                    0.000000
NUM_UNITS             0.422989
ROOMS                 0.000000
BEDRM                 0.000000
AYB                   0.000000
YR_RMDL               0.413240
EYB                   0.000000
STORIES               0.423268
SALEDATE              0.000000
PRICE                 0.000000
QUALIFIED             0.000000
SALE_NUM              0.000000
GBA                   0.422989
BLDG_NUM              0.000000
STYLE                 0.422989
STRUCT                0.422989
GRADE                 0.422989
CNDTN                 0.422989
EXTWALL               0.422989
ROOF                  0.422989
INTWALL               0.422989
KITCHENS              0.423001
FIREPLACES            0.000000
USECODE               0.000000
LANDAREA              0.000000
GIS_LAST_MOD_DTTM     0.000000
SOURCE                0.000000
LIVING_GBA            0.577011
ZIPCODE               0.000000
LATITUDE              0.000000
LONGITUDE             0.000000
ASSESSME

In [ ]:
#drop null from SALEDATE
odatd = odatd.dropna(subset=['SALEDATE'])

In [ ]:
#create categorical and numerical dataframes
catcol = []
intcol = []
for col in odatd.columns:
    if (odatd[col].dtype == object):
        catcol.append(col)
    if (odatd[col].dtype == np.int64) or (odatd[col].dtype == np.float64):
        numcol.append(col)
catdf = odatd[catcol].astype('category')
numdf = odatd[numcol].astype(np.float64)

In [ ]:
#using Latitute and Longitude to sort 
tsortd = odatd.sort_values(['LATITUDE', 'LONGITUDE'])

In [ ]:
#fill using rolling mean
#now, get the new frame to just have the na values filled with the 
#odatd[col].rolling(11, center=True, min_periods=1).mean()
dfill = pd.DataFrame()
for col in numdf.columns:
    dfill[col] = tsortd[col].fillna(tsortd[col].rolling(301, center=True, min_periods=1).mean())
print(missingdat(tsortd))
print(missingdat(dfill))

In [105]:
#create df with mode values for groupby
modedf = pd.DataFrame()
for col in catdf.columns:
    modedf[col] = tsortd.groupby('ASSESSMENT_NBHD')[col].apply(lambda x: x.mode())
modedf.head()

,,HEAT,AC,SALEDATE,QUALIFIED,STYLE,STRUCT,GRADE,CNDTN,EXTWALL,ROOF,INTWALL,GIS_LAST_MOD_DTTM,SOURCE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_BLOCK,WARD,SQUARE,QUADRANT
ASSESSMENT_NBHD,,,,,,,,,,,,,,,,,,,,
16th Street Heights,0,Hot Water Rad,Y,2016-08-18 00:00:00,Q,2 Story,Single,Good Quality,Good,Common Brick,Metal- Sms,Hardwood,2018-07-22 18:01:43,Residential,16th Street Heights,049 A 16th Street Heights,002001 2010,Ward 4,2798,NW
American University,0,Warm Cool,Y,2005-09-30 00:00:00,Q,2 Story,Single,Good Quality,Good,Common Brick,Slate,Hardwood,2018-07-22 18:01:43,Residential,American University,001 B American University,001001 1013,Ward 3,1730,NW
Anacostia,0,Forced Air,Y,2013-07-16 00:00:00,Q,2 Story,Row Inside,Average,Average,Common Brick,Metal- Sms,Hardwood,2018-07-22 18:01:43,Residential,Anacostia,002 B Anacostia,007504 1004,Ward 8,5807,SE
Barry Farms,0,Forced Air,Y,2006-02-08 00:00:00,Q,2 Story,Row Inside,Average,Average,Common Brick,Built Up,Hardwood,2018-07-22 18:01:38,Condominium,Barry Farms,NaN,007407 2008,Ward 8,5869,SE
Berkley,0,Warm Cool,Y,2014-06-27 00:00:00,Q,2 Story,Single,Very Good,Good,Common Brick,Comp Shingle,Hardwood,2018-07-22 18:01:43,Residential,Berkley,004 A Berkley,000802 1001,Ward 3,1368,NW


In [ ]:
#HOW DO I USE THE MODE VALUES IN THIS TABLE TO FILL THE NA VALUES IN THE DATASET 
#BY REFERENCING INDEX AND COLUMN?
#I thought the solution below would work...? 

In [110]:
#why didn't this work?!
for col in catdf.columns:
    dfill[col] = tsortd.groupby("ASSESSMENT_NBHD")[col].transform(lambda x: x.fillna(x.mode()))  
print(missingdat(odatd[catcol]))
print(missingdat(dfill[catcol]))

HEAT                  0.000000
AC                    0.000000
SALEDATE              0.000000
QUALIFIED             0.000000
STYLE                 0.422989
STRUCT                0.422989
GRADE                 0.422989
CNDTN                 0.422989
EXTWALL               0.422989
ROOF                  0.422989
INTWALL               0.422989
GIS_LAST_MOD_DTTM     0.000000
SOURCE                0.000000
ASSESSMENT_NBHD       0.000000
ASSESSMENT_SUBNBHD    0.195839
CENSUS_BLOCK          0.426124
WARD                  0.000000
SQUARE                0.000000
QUADRANT              0.000000
dtype: float64
None
HEAT                  0.000000
AC                    0.000000
SALEDATE              0.000000
QUALIFIED             0.000000
STYLE                 0.422989
STRUCT                0.422989
GRADE                 0.422989
CNDTN                 0.422989
EXTWALL               0.422989
ROOF                  0.422989
INTWALL               0.422989
GIS_LAST_MOD_DTTM     0.000000
SOURCE             